In [1]:
import numpy as np
import skystar as ss
# import matplotlib.pyplot as plt
from skystar.model import Simple_RNN
from skystar.dataloader import SeqDataloader
from skystar.dataset import Dataset
from skystar.optimizer import Adam
from skystar.model import Better_RNN
from skystar.core import mean_squared_error

def sin(x):
    y=np.sin(x)
    return y

'''生成sin数据'''
timestep=np.arange(0,999,1)#999个数据
data=sin(np.linspace(0,4*np.pi,1000))+ np.random.normal(-0.025, 0.025,size=1000)

x_train=data[0:999]
y_train=data[1:1000]#表示x_train下一时间步的数据
data

array([-9.8133e-04, -3.7956e-02, -4.3904e-02,  1.2464e-02,  4.5043e-02,
       -1.2571e-02,  2.1952e-02,  1.5845e-02,  1.0536e-01,  1.2172e-01,
        7.0455e-02,  9.3854e-02,  1.0720e-01,  1.0413e-01,  1.4942e-01,
        1.7627e-01,  1.4915e-01,  2.2733e-01,  2.2234e-01,  2.2729e-01,
        2.6580e-01,  2.3040e-01,  2.3307e-01,  2.7124e-01,  2.4568e-01,
        3.0646e-01,  2.8971e-01,  2.6937e-01,  3.4156e-01,  3.0305e-01,
        3.0943e-01,  3.2029e-01,  3.7637e-01,  3.8584e-01,  3.4900e-01,
        4.3235e-01,  4.4425e-01,  4.1060e-01,  4.6425e-01,  5.0405e-01,
        4.5236e-01,  4.4760e-01,  5.0749e-01,  4.4093e-01,  4.7523e-01,
        5.0711e-01,  4.9284e-01,  4.7022e-01,  5.5060e-01,  5.5771e-01,
        5.6764e-01,  5.5905e-01,  5.4896e-01,  5.8991e-01,  6.4434e-01,
        5.6472e-01,  5.8911e-01,  6.1739e-01,  6.2375e-01,  6.2414e-01,
        6.7413e-01,  6.4151e-01,  6.5725e-01,  7.1551e-01,  6.8772e-01,
        6.8700e-01,  7.1796e-01,  7.9105e-01,  7.2120e-01,  7.42

In [2]:
'''查看数据'''
# plt.plot(timestep,x_train)
# plt.plot(timestep,y_train)
# plt.show()

'查看数据'

In [3]:
'''创建序列数据集'''
class Sin_dataset(Dataset):#创建数据集
    def __init__(self,training):
        super(Sin_dataset,self).__init__(training=training)
    def prepare(self):
        if self.training:
            self.data=x_train
            self.label=y_train
train = Sin_dataset(training=True)
data_train=SeqDataloader(train,batch_size=30)

In [4]:
'''模型训练'''
model=Better_RNN(300,1)
model.to_gpu()
data_train.to_gpu()
epoch=100
lr=0.01
optimizer=Adam(lr).setup(model)
# for x in range(epoch):
#     model.reset_state()
#     loss,count=0,0
#     for x,t in data_train:
#         y=model(x)
#         loss+=mean_squared_error(y,t)
#         count+=1
#         if count>=len(data_train):
#             print(f"count:{count}   len:{len(data_train)}")
#             print("===============================")
#         if count%30==0 or count==len(data_train):#每30个数据截断连接并反向传播
#             model.cleangrads()
#             loss.backward()
#             loss.unchain_backward()
#             optimizer.update()
#     avg_loss=loss/count
#     print(f'Train_Loss {avg_loss}')
model.TrainForSeq(data_train,epoch=epoch,optimizer=Adam)

========Setting mode to Training========
===============Training Begin===============
GPU is available, and all parameters are converted to cp.ndarray
Epoch 1:


Training:   0%|          | 0/34 [00:00<?, ?it/s]


ValueError: iter size must be 2 or 3

In [ ]:
'''生成一个cos序列，并预测'''
model.reset_state()
xs=np.cos(np.linspace(0,4*np.pi,1000))
out=[]
model.to_cpu()
count=0
with ss.no_grad():
    for x in xs:
        count += 1
        x = ss.Variable(np.array(x).reshape(1, 1))
        y = model(x)
        y.data = y.data.squeeze()
        out.append(y.data)
len(out)
model.save_to_onnx(x)

In [ ]:
'''预测结果查看'''
# list=np.arange(0,1000,1)
# plt.plot(list,xs,label='cosx')
# plt.plot(list,out,label='predict')
# plt.legend()
# plt.show()